In [1]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.2.0 pyspark-shell'

In [2]:
#    Spark
from pyspark import SparkContext  
#    Spark Streaming
from pyspark.streaming import StreamingContext  
#    Kafka
from pyspark.streaming.kafka import KafkaUtils  
#    json parsing
import json

In [3]:
sc = SparkContext(appName="PythonSparkStreamingKafka_RM_01")  
sc.setLogLevel("WARN")

In [7]:
ssc = StreamingContext(sc, 2) 
topic = "test"
brokers = "172.17.0.1:9092"

In [8]:
kvs = KafkaUtils.createDirectStream(ssc, [topic], {"metadata.broker.list": brokers})

In [ ]:
lines = kvs.map(lambda x: x[1])
lines.pprint()
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2017-11-24 22:43:44
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:43:46
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:43:48
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:43:50
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:43:52
-------------------------------------------
raw_bytes
raw_bytes
raw_bytes
raw_bytes

-------------------------------------------
Time: 2017-11-24 22:43:54
-------------------------------------------
raw_bytes
raw_bytes
raw_bytes
raw_bytes
raw_bytes
raw_bytes
raw_bytes

-------------------------------------------
Time: 2017-11-24 22:43:56
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:43:58
---------------

-------------------------------------------
Time: 2017-11-24 22:46:06
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:46:08
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:46:10
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:46:12
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:46:14
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:46:16
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:46:18
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:46:20
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:46:22
----------

-------------------------------------------
Time: 2017-11-24 22:48:30
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:48:32
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:48:34
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:48:36
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:48:38
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:48:40
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:48:42
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:48:44
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:48:46
----------

-------------------------------------------
Time: 2017-11-24 22:50:54
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:50:56
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:50:58
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:51:00
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:51:02
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:51:04
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:51:06
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:51:08
-------------------------------------------

-------------------------------------------
Time: 2017-11-24 22:51:10
----------